# Stock Analysis Notebook

This Jupyter notebook is designed to analyze stocks based on various filters and sentiment analysis of news articles. 

## Dependencies Installation

First, we need to install the necessary dependencies. Uncomment the lines below if you haven't installed these packages yet.



In [21]:
'''
%pip install finvizfinance
%pip install pandas
%pip install transformers
%pip install yfinance
%pip install goose3
%pip install requests
%pip install ipywidgets
%pip install torch
%pip install tensorflow
%pip install nltk
import nltk
nltk.download('punkt')
'''

"\n%pip install finvizfinance\n%pip install pandas\n%pip install transformers\n%pip install yfinance\n%pip install goose3\n%pip install requests\n%pip install ipywidgets\n%pip install torch\n%pip install tensorflow\n%pip install nltk\nimport nltk\nnltk.download('punkt')\n"



## Import Libraries

Next, we import the necessary libraries for our analysis.



In [22]:
# Standard library imports
from datetime import datetime, timedelta
import warnings
from requests.exceptions import HTTPError

from finvizfinance.screener.overview import Overview # type: ignore
from finvizfinance.quote import finvizfinance        # type: ignore
from IPython.display import display                  # type: ignore
import pandas as pd                                  # type: ignore
from transformers import pipeline                    # type: ignore
import yfinance as yf                                # type: ignore
from goose3 import Goose                             # type: ignore
from requests import get                             # type: ignore
from nltk.tokenize import sent_tokenize              # type: ignore
from transformers import AutoTokenizer               # type: ignore
from bs4 import BeautifulSoup                        # type: ignore
import csv
import os
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_colwidth', None) # Display full text in pandas dataframe / no line wrapping



## Stock Filtering

We define our filters for selecting stocks. These filters can be modified as per your requirements.



In [23]:
FILTERS_DICT = {
    'Performance': 'Today +10%',     # Day increase 10%
    'Relative Volume': 'Over 5',       # High Relative Volume
    'Price': 'Under $20',              # Price under 20 USD
    'Float': 'Under 10M'               # Float under 10 million
}

NEWS_AGE_LIMIT_IN_DAYS = 5 # Limit of days for the news to be considered relevant


# Alternative filtering to consider:
'''
FILTERS_DICT = {'Debt/Equity':'Under 1',                 # Positive Operating Margin
                'PEG':'Low (<1)',                        # Debt-to-Equity ratio under 1
                'Operating Margin':'Positive (>0%)',     # Low P/B (under 1)
                'P/B':'Low (<1)',                        # Low P/E ratio (under 15)
                'P/E':'Low (<15)',                       # Low PEG ratio (under 1)
                'InsiderTransactions':'Positive (>0%)<'} # Positive Insider Transactions
'''


# The filters and general manual link for the finvizfinance library: https://finvizfinance.readthedocs.io/_/downloads/en/latest/pdf/ 
# Possible filters can be found by running the following code:

#from finvizfinance.screener.overview import Overview # type: ignore
#foverview = Overview()    # Create Overview object
#foverview.get_filters()   # Get list of all possible filters

# And after to see the possible options for a filter, run the following code:
#foverview.get_filter_options('Relative Volume') # Get list of all possible options for a filter, example on 'Relative Volume'


"\nFILTERS_DICT = {'Debt/Equity':'Under 1',                 # Positive Operating Margin\n                'PEG':'Low (<1)',                        # Debt-to-Equity ratio under 1\n                'Operating Margin':'Positive (>0%)',     # Low P/B (under 1)\n                'P/B':'Low (<1)',                        # Low P/E ratio (under 15)\n                'P/E':'Low (<15)',                       # Low PEG ratio (under 1)\n                'InsiderTransactions':'Positive (>0%)<'} # Positive Insider Transactions\n"



## Fetching Filtered Stocks

We define a function `get_filtered_stocks()` to fetch the stocks that meet our filter criteria.



In [24]:
def get_filtered_stocks():
    """
    Returns a list of tickers with:

    """
    
    foverview = Overview()
    foverview.set_filter(filters_dict=FILTERS_DICT)
    df_overview = foverview.screener_view()
    if not os.path.exists('out'): #ensures you have an 'out' folder ready
        os.makedirs('out')
    df_overview.to_csv('out/Overview.csv', index=False)
    
    tickers = df_overview['Ticker'].to_list()
    display(df_overview)
    return tickers



## Sentiment Analysis

We define a function `get_ticker_news_sentiment(ticker)` to perform sentiment analysis on the news articles of a given ticker.



In [25]:

def get_recent_ticker_news(ticker):
    # Get news articles
    yf_ticker = yf.Ticker(ticker)
    
    try:
        news_list = yf_ticker.get_news()
    except:
        print(f"Error getting news for ticker {ticker}")
        return
    
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    extractor = Goose()
    
    titles, links, times, texts = [], [], [], []

    for dic in news_list:        
        try:
            response = get(dic['link'], headers=headers)
            response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')  # Python 3.6
        except Exception as err:
            print(f'Other error occurred: {err}')  # Python 3.6
        else:
            pass
        
        # Append the values to the respective lists
        titles.append(dic['title'])                                             # Article title
        links.append(dic['link'])                                               # Article link                         
        times.append(datetime.fromtimestamp(dic['providerPublishTime']))        # Article publish time
        texts.append(extractor.extract(raw_html=response.content).cleaned_text) # Article text
    
    news_df = pd.DataFrame({'time': times, 'title': titles, 'text': texts, 'link': links})
    
    # Filter out news older than ""news_age_limit_in_days" days
    news_df = news_df[news_df['time'] > datetime.now() - timedelta(days=NEWS_AGE_LIMIT_IN_DAYS)]
    
    return news_df



def get_ticker_news_sentiment(stocknews):
    ALLOW_TOKENIZATION = True # True: the model will feed the full article into the model in chunks of 512 tokens, 
    #                            False: the model will consider only the first sentences of the article until the total number of tokens does not exceed 512

    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    pipe = pipeline("text-classification", model="ProsusAI/finbert")

    # Initialize a list for the sentiment scores
    sentiments = []
    sentiment_scores = []

    for _, row in stocknews.iterrows():
        text = row['text']

        # Model max input is 512 tokens, so we need to split the text into chunks of 512 tokens
        if ALLOW_TOKENIZATION: # feed into model in chunks
            inputs = tokenizer.encode_plus(
                text,
                max_length=510,
                truncation='longest_first',
                padding='max_length',
                return_tensors='pt',
            )

            input_ids = inputs["input_ids"].tolist()[0]
            new_text = tokenizer.decode(input_ids)
        else: # feed into model only the first sentences (until 512 tokens)
            sentences = sent_tokenize(text)
            new_text = ''
            for sentence in sentences:
                if len(tokenizer.encode(new_text)) + len(tokenizer.encode(sentence)) <= 512:
                    new_text += ' ' + sentence
                else:
                    break

        results = pipe(new_text) # run the model

        # Get the sentiment score from the results and append it to the list        
        sentiment = results[0]['label']
        sentiment_score = results[0]['score'] if sentiment == 'positive' else -results[0]['score']
        
        sentiments.append(sentiment)
        sentiment_scores.append(sentiment_score) # instead assigning positive etc values, assign the score

    # Add the sentiment scores to the DataFrame
    stocknews['sentiment'] = sentiments
    stocknews['sentiment_score'] = sentiment_scores

    return stocknews






## Generate CSV

We define a function `generate_csv(sentiment,ticker)` to generate a CSV file for each ticker's sentiment analysis.



In [26]:
def generate_csv(stocknews, ticker):
    # Select all columns except 'text'
    stocknews = stocknews.loc[:, stocknews.columns != 'text']
    stocknews.to_csv(f'out/{ticker}.csv', index=False)



## Fetch Sentiments and Generate CSVs

We fetch the sentiments for each undervalued stock and generate a CSV file for each.



In [28]:
undervalued = get_filtered_stocks()

stocknews = []
for ticker in undervalued:
    stocknews = get_recent_ticker_news(ticker)
    stocknews = get_ticker_news_sentiment(stocknews)

    if stocknews is not None:
        #stocknews.append(stocknews)
        generate_csv(stocknews,ticker)
    else:
        print(f'No news found for {ticker}')
    print(f'{ticker} News sentiment analysis done, {len(undervalued) - undervalued.index(ticker) - 1} stock tickers left')	


,Ticker,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
0,AWIN,AERWINS Technologies Inc,Technology,Scientific & Technical Instruments,USA,5780000.0,None,6.28,1.3173,48593066.0
1,EDBL,Edible Garden AG Inc,Consumer Defensive,Farm Products,USA,2720000.0,None,4.94,0.1569,18543322.0
2,IKT,Inhibikase Therapeutics Inc,Healthcare,Biotechnology,USA,12900000.0,None,1.99,0.2922,564724.0
3,JFBR,Jeffs Brands Ltd,Consumer Cyclical,Internet Retail,Israel,360000.0,None,0.30,0.3254,5990676.0
4,LIDR,AEye Inc,Technology,Software - Infrastructure,USA,21710000.0,None,3.34,0.3577,35081991.0
5,MYSZ,My Size Inc,Technology,Software - Application,Israel,3400000.0,None,5.32,0.1247,345021.0
6,RELI,Reliance Global Group Inc,Financial,Insurance Brokers,USA,2100000.0,None,0.37,0.3654,26781763.0
7,SOWG,Sow Good Inc,Consumer Defensive,Packaged Foods,USA,156970000.0,None,15.76,0.4185,381985.0
8,SYTA,Siyata Mobile Inc,Technology,Communication Equipment,Canada,2120000.0,None,3.48,0.1154,830533.0
9,TTOO,T2 Biosystems Inc,Healthcare,Diagnostics & Research,USA,52480000.0,None,5.97,0.4351,3117939.0


AWIN News sentiment analysis done, 9 stock tickers left
EDBL News sentiment analysis done, 8 stock tickers left




## Display Sentiments

Finally, we display the sentiments for each ticker.



In [ ]:
for i in range(len(sentiments)):
    display(sentiments[i])

,Ticker,Date,Article title,Article sentiment
0,AWIN,2024-04-23T21:15:00.000Z,AERWINS Technologies Announces Additional Staff Determination,neutral
1,AWIN,2024-03-28T15:25:00.000Z,AERWINS Technologies Announces 1-For-100 Reverse Stock Split,neutral


,Ticker,Date,Article title,Article sentiment
0,BBLG,2024-03-06T21:15:00.000Z,Bone Biologics Announces Closing of $2.0 Million Public Offering,neutral
1,BBLG,2024-03-04T14:15:00.000Z,Bone Biologics Announces Pricing of $2.0 Million Public Offering,neutral
2,BBLG,2024-03-01T13:00:00.000Z,Bone Biologics Reports Progress With NB1 Clinical Program,neutral
3,BBLG,2024-02-22T11:14:00.000Z,BBLG Releases Annual Report Amid Continued Pursuit of Breakthrough Treatment,neutral


,Ticker,Date,Article title,Article sentiment
0,COCP,2024-05-13T12:00:00.000Z,Cocrystal Pharma Reports First Quarter 2024 Financial Results and Provides Updates on its Antiviral Drug-Development Programs,neutral
1,COCP,2024-05-01T12:00:00.000Z,Enrollment Completed in Phase 2a Study with Cocrystal Pharma’s Oral Antiviral Candidate CC-42344 for Pandemic and Seasonal Influenza,neutral
2,COCP,2024-04-23T14:10:00.000Z,COCP: Topline Data for Influenza A and Dual Coronavirus/Norovirus Programs Expected in 2024; Model Revision Leads to Valuation of $6 Per Share…,neutral
3,COCP,None,Cocrystal Pharma Reports 2023 Financial Results and Provides Updates on its Antiviral Drug-Development Programs,neutral
4,COCP,2024-03-19T12:00:00.000Z,Cocrystal Pharma Receives Pre-IND Responses from the FDA on Oral CC-42344 for Treating Influenza A,neutral
5,COCP,2024-03-05T13:30:00.000Z,Cocrystal Pharma Discusses Antiviral Therapeutics' Clinical Development and 2024 Value Catalysts in New CEO Video Interview on Planet MicroCap,neutral
6,COCP,2024-02-20T13:40:00.000Z,COCP: Cocrystal Pharma provides updates on its clinical programs.,neutral


,Ticker,Date,Article title,Article sentiment
0,EDBL,2024-05-15T11:00:00.000Z,Edible Garden Reports Significant 27.6% Year-Over-Year Revenue Growth for First Quarter of 2024,neutral
1,EDBL,2024-05-08T13:00:00.000Z,Edible Garden Schedules First Quarter 2024 Financial Results and Business Update Conference Call,neutral
2,EDBL,2024-05-07T13:00:00.000Z,Edible Garden Announces Product Development Agreement with Hermann Pickle Company,positive
3,EDBL,2024-04-29T13:00:00.000Z,Edible Garden Begins Shipping ‘Garden Starters’ in Time for Spring Planting Season,positive
4,EDBL,2024-04-23T13:00:00.000Z,"EY Announces Jim Kras, CEO of Edible Garden AG Incorporated as an Entrepreneur Of The Year® 2024 New Jersey Award Finalist",neutral
5,EDBL,2024-04-22T13:15:00.000Z,Edible Garden Regains Compliance with Nasdaq Bid Price Requirement,neutral
6,EDBL,2024-04-17T12:00:00.000Z,Edible Garden Reports Preliminary First Quarter Year-Over-Year Increase in Produce Revenues of 40%; Cut Herb Revenue Increased More Than 200% for the Same Period,neutral
7,EDBL,2024-04-11T13:00:00.000Z,Edible Garden Announces Pulp Sustainable Line of Products Now Available Through UNFI Distributors,neutral


,Ticker,Date,Article title,Article sentiment
0,IKT,2024-05-15T12:45:00.000Z,Inhibikase Therapeutics Announces Request for Withdrawal of S-1 Registration Statement,neutral
1,IKT,2024-05-09T12:00:00.000Z,Inhibikase Therapeutics Announces Final Pre-IND Meeting Outcomes for IkT-001Pro as a Treatment for Pulmonary Arterial Hypertension,neutral
2,IKT,2024-04-18T20:30:00.000Z,Inhibikase Therapeutics Issues Letter to Shareholders and Provides Update on Development Programs,neutral
3,IKT,2024-04-03T12:05:00.000Z,Inhibikase Therapeutics Announces Pre-IND Meeting with the FDA for IkT-001Pro in Pulmonary Arterial Hypertension,neutral
4,IKT,2024-04-02T15:51:27.000Z,Inhibikase Therapeutics Full Year 2023 Earnings: EPS Misses Expectations,neutral
5,IKT,2024-04-02T14:25:22.000Z,"Inhibikase Therapeutics, Inc. (NASDAQ:IKT) Q4 2023 Earnings Call Transcript",neutral
6,IKT,2024-03-29T01:06:00.000Z,Q4 2023 Inhibikase Therapeutics Inc Earnings Call,neutral
7,IKT,2024-03-27T20:15:00.000Z,Inhibikase Therapeutics Reports Fourth Quarter and Full Year 2023 Financial Results and Highlights Recent Activity,neutral


...

,Ticker,Date,Article title,Article sentiment
0,MYSZ,2024-05-07T14:08:00.000Z,My Size Regains Compliance with Nasdaq Minimum Closing Bid Price Rule,positive
1,MYSZ,2024-04-15T20:15:00.000Z,My Size Announces Reverse Stock Split,neutral
2,MYSZ,2024-04-04T11:03:49.000Z,My Size Full Year 2023 Earnings: US$2.50 loss per share (vs US$7.47 loss in FY 2022),neutral
3,MYSZ,2024-02-22T14:00:00.000Z,"Naiz Fit, a My Size Inc Company, Unveils Groundbreaking Generative AI Strategies to Revolutionize Fashion Tech",neutral


,Ticker,Date,Article title,Article sentiment
0,RELI,2024-05-15T12:30:00.000Z,Reliance Global Group Announces Game-Changing Acquisition of Spetner Associates; Projected to Double Company’s Revenue,neutral
1,RELI,2024-04-23T11:30:00.000Z,Reliance Global Group Inc.: Invitation to EF Hutton Annual Global Conference,neutral
2,RELI,2024-04-04T13:30:00.000Z,"New to The Street TV Announces its Next Two Shows Broadcastings with Five Business Guest Interviews, Episode 566 Airs on Bloomberg TV as A Sponsored Programming Tonight, Thursday, April 4, 2024, at 9:30 PM PT, and Episode 567 Airs on The FOX Business Network, Monday, April 8, 2024, at 10:30 PM PT",neutral
3,RELI,2024-04-04T12:00:00.000Z,"Reliance Global Group’s Revenue Grows by 17% and 40% in 2023 Compared to 2022 and 2021, respectively",neutral
4,RELI,2024-04-03T12:30:00.000Z,Reliance Global Group Schedules Fourth Quarter 2023 Financial Results and Business Update Conference Call,neutral


,Ticker,Date,Article title,Article sentiment
0,SNTG,2024-04-28T13:13:24.000Z,Sentage Holdings Full Year 2023 Earnings: US$0.80 loss per share (vs US$1.08 loss in FY 2022),neutral


,Ticker,Date,Article title,Article sentiment
0,SOWG,2024-05-15T12:30:00.000Z,Sow Good Reports Strong First Quarter 2024 Results,neutral
1,SOWG,2024-05-09T20:00:00.000Z,Sow Good Announces Exercise of Overallotment Option from Public Offering,neutral
2,SOWG,2024-05-08T12:30:00.000Z,"Sow Good to Hold First Quarter 2024 Conference Call on Wednesday, May 15, 2024 at 10:00 a.m. ET",neutral
3,SOWG,2024-05-02T12:41:00.000Z,Sow Good Announces Nasdaq Uplisting and Pricing of $12 Million Public Offering,neutral
4,SOWG,2024-04-16T13:22:00.000Z,Sow Good Announces Proposed Public Offering of Common Stock,neutral
5,SOWG,2024-04-02T21:40:00.000Z,Sow Good Appoints Brendon Fischer as Interim Chief Financial Officer,neutral
6,SOWG,2024-03-28T22:59:00.000Z,Sow Good Announces $3.7M Private Placement,neutral
7,SOWG,2024-03-23T14:12:32.000Z,Sow Good Inc. (PNK:SOWG) Q4 2023 Earnings Call Transcript,neutral


,Ticker,Date,Article title,Article sentiment
0,TTOO,2024-05-15T11:00:00.000Z,T2 Biosystems Announces $8 Million Private Placement Priced At-The-Market Under Nasdaq Rules,neutral
1,TTOO,2024-05-14T21:00:00.000Z,T2 Biosystems Announces CRG’s Conversion from Preferred Stock to Common Stock,neutral
2,TTOO,2024-05-07T13:00:00.000Z,T2 Biosystems Announces Letter of Intent to Enter Strategic Partnership for Lyme Disease,neutral
3,TTOO,2024-05-07T12:48:54.000Z,"T2 Biosystems, Inc. (NASDAQ:TTOO) Q1 2024 Earnings Call Transcript",neutral
4,TTOO,2024-05-07T10:11:13.000Z,Q1 2024 T2 Biosystems Inc Earnings Call,neutral
5,TTOO,2024-05-07T07:09:12.000Z,T2 Biosystems Inc (TTOO) Q1 2024 Earnings Call Transcript Highlights: Revenue Growth Amidst ...,neutral
6,TTOO,2024-05-06T20:45:03.000Z,T2 Biosystems: Q1 Earnings Snapshot,neutral
7,TTOO,2024-05-06T20:34:00.000Z,T2 Biosystems Announces First Quarter 2024 Financial Results,neutral




## Additional Code

The following code snippets are not integrated into the main flow of the notebook but can be used for additional analysis.



In [ ]:
'''
quote = finvizfinance('SGE')
df = quote.ticker_inside_trader()
from datetime import datetime
today = datetime.today().date()
df = quote.ticker_news()
df = df[df['Date'].dt.date == today]
df
df = quote.ticker_fundament()
df
'''

"\nquote = finvizfinance('SGE')\ndf = quote.ticker_inside_trader()\nfrom datetime import datetime\ntoday = datetime.today().date()\ndf = quote.ticker_news()\ndf = df[df['Date'].dt.date == today]\ndf\ndf = quote.ticker_fundament()\ndf\n"